### Importing trip start times, to be used in calculations of both nominal and vulnerable data

In [1]:
trip_file = open('../trips/trip_combined.xml', 'r')
from xml.etree import ElementTree as ET
tree = ET.parse(trip_file)
root = tree.getroot()
start_times = {}
for trip in root:
    start_times[trip.attrib['id']] = int(trip.attrib['depart'])

### Importing travel times for nominal and vul

In [1]:
import json
#with open('../output/net_dump/lmbd8/traveltime_55_1_28800_57600_8_False.json') as f:
with open('../output/net_dump/lmbd100/traveltime_55_1_28800_57600_100_False.json') as f:
    high_vul_tt = json.load(f)

#with open('../output/net_dump/lmbd100/traveltime_1_1_0_0_100_True.json') as f:
with open('../output/net_dump/lmbd100/traveltime_55_1_0_0_100_True.json') as f:
    nom_tt = json.load(f)
start_times_nom = {}
start_times_vul = {}

### Calculate end time for each trip for nominal and vul

In [2]:
end_nom = {}
end_high_vul = {}
unequal_trips = 0
for trip in nom_tt:
    if trip != 'sim_time':
        start_times_nom[trip] = int(nom_tt[trip][1])
        start_times_vul[trip] = int(high_vul_tt[trip][1])
        end_nom[trip] = int(nom_tt[trip][0])
        end_high_vul[trip] = int(high_vul_tt[trip][0])


### Count the number of trips that END at each time step

In [3]:
from collections import defaultdict
nom_trips_left = defaultdict(int)
vul_trips_left = defaultdict(int)
for trip in end_nom:
    nom_trips_left[end_nom[trip]] += 1
    vul_trips_left[end_high_vul[trip]] += 1

### Create a time series for trips completed for nominal and vul

In [4]:
nom_trip_vec = []
total_trips = len(end_nom)

for t in range(max(nom_trips_left.keys())):
    if t in nom_trips_left:
        total_trips = total_trips - nom_trips_left[t]
    nom_trip_vec.append(total_trips)

In [5]:
vul_trip_vec = []
total_trips = len(end_high_vul)
for t in range(max(vul_trips_left.keys())):
    if t in vul_trips_left:
        total_trips = total_trips - vul_trips_left[t]
    vul_trip_vec.append(total_trips)

In [6]:
from matplotlib import pyplot as plt
%matplotlib 

Using matplotlib backend: Qt5Agg


### Create plots for nominal and vulnerable trips completed

In [16]:
plt.plot(nom_trip_vec, 'b', linewidth=2)
plt.plot(vul_trip_vec, 'r--', linewidth=2)

### Count trips currently on network for nominal and vul

In [13]:
nom_current_trips = defaultdict(int)
vul_current_trips = defaultdict(int)
for trip in end_nom:
    for t in range(start_times_nom[trip], end_nom[trip]):
        nom_current_trips[t] += 1
for trip in end_high_vul:
    for t in range(start_times_vul[trip], end_high_vul[trip]):
        vul_current_trips[t] += 1

nom_curr_trips_vec = []
vul_curr_trips_vec = []
for t in range(max(nom_current_trips.keys())):
    if t in nom_current_trips:
        nom_curr_trips_vec.append(nom_current_trips[t])
    else:
        nom_curr_trips_vec.append(0)

for t in range(max(vul_current_trips.keys())):
    if t in vul_current_trips:
        vul_curr_trips_vec.append(vul_current_trips[t])
    else:
        vul_curr_trips_vec.append(0)



In [17]:
plt.plot(nom_curr_trips_vec, 'b', linewidth=2)
plt.plot(vul_curr_trips_vec, 'r--', linewidth=2)